In [ ]:
import sqlite3
import yt_dlp
from sentence_transformers import SentenceTransformer
import json
import time
from youtube_transcript_api import YouTubeTranscriptApi
import faiss
import numpy as np

# ==========================
# PART 1: Extract Video Metadata and Compute Description Embeddings
# ==========================
channel_url = "https://www.youtube.com/c/MediaEventsca/videos"
ydl_opts = {
    'skip_download': True,
    'ignoreerrors': True,
    'quiet': True,    
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(channel_url, download=False)

videos_data = []
for entry in info.get('entries', []):
    if entry and 'id' in entry:
        video_id = entry['id']
        url = f"https://www.youtube.com/watch?v={video_id}"
        description = entry.get('description', "")
        videos_data.append((video_id, url, description))

model = SentenceTransformer('all-MiniLM-L6-v2')
videos_with_embedding = []
for video_id, url, description in videos_data:
    if description.strip():
        embedding = model.encode(description)
        embedding_json = json.dumps(embedding.tolist())
    else:
        embedding_json = None
    videos_with_embedding.append((video_id, url, description, embedding_json))

conn = sqlite3.connect('videos.db', timeout=30)
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS videos")
cursor.execute('''
CREATE TABLE videos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    video_id TEXT UNIQUE,
    url TEXT,
    description TEXT,
    description_embedding TEXT
)
''')
for video in videos_with_embedding:
    video_id, url, description, description_embedding = video
    try:
        cursor.execute(
            'INSERT INTO videos (video_id, url, description, description_embedding) VALUES (?, ?, ?, ?)', 
            (video_id, url, description, description_embedding)
        )
    except sqlite3.IntegrityError:
        cursor.execute(
            'UPDATE videos SET url = ?, description = ?, description_embedding = ? WHERE video_id = ?',
            (url, description, description_embedding, video_id)
        )
conn.commit()
conn.close()

print(f"Extracted and stored {len(videos_with_embedding)} videos with description embeddings.")

# ==========================
# PART 2: Retrieve Transcripts, Chunk Text, and Store in Captions Table
# ==========================
def chunk_text(text, chunk_size=30):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

conn = sqlite3.connect('videos.db', timeout=30)
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS captions")
cursor.execute('''
CREATE TABLE captions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    video_id TEXT,
    chunk_order INTEGER,
    text TEXT,
    embedding TEXT
)
''')
conn.commit()

cursor.execute("SELECT video_id FROM videos")
all_video_ids = [row[0] for row in cursor.fetchall()]

for vid in all_video_ids:
    try:
        transcript = YouTubeTranscriptApi.get_transcript(vid)
        full_text = " ".join([entry.get('text', "") for entry in transcript])
        chunks = chunk_text(full_text, chunk_size=30)
        for order, chunk in enumerate(chunks):
            inserted = False
            while not inserted:
                try:
                    cursor.execute(
                        "INSERT INTO captions (video_id, chunk_order, text) VALUES (?, ?, ?)",
                        (vid, order, chunk)
                    )
                    inserted = True
                except sqlite3.OperationalError as e:
                    if "database is locked" in str(e):
                        time.sleep(0.5)
                    else:
                        raise e
        conn.commit()
    except Exception as e:
        print(f"Could not retrieve transcript for video {vid}: {e}")

print("Finished retrieving transcripts and storing caption chunks.")
conn.close()

# ==========================
# PART 3: Compute Caption Embeddings and Update Captions Table
# ==========================
conn = sqlite3.connect('videos.db', timeout=30)
cursor = conn.cursor()
cursor.execute("SELECT id, text FROM captions WHERE embedding IS NULL")
rows = cursor.fetchall()
print("Computing embeddings for", len(rows), "caption chunks.")
batch_size = 128
for i in range(0, len(rows), batch_size):
    batch = rows[i:i+batch_size]
    ids = [r[0] for r in batch]
    texts = [r[1] for r in batch]
    embeddings = model.encode(texts, batch_size=batch_size, show_progress_bar=True)
    for id_val, emb in zip(ids, embeddings):
        emb_json = json.dumps(emb.tolist())
        cursor.execute("UPDATE captions SET embedding=? WHERE id=?", (emb_json, id_val))
    conn.commit()
conn.close()

# ==========================
# PART 4: Build FAISS Index from Caption Embeddings
# ==========================
def load_caption_embeddings(batch_size=1000):
    conn = sqlite3.connect('videos.db', timeout=30)
    c = conn.cursor()
    c.execute("SELECT id, embedding, video_id, chunk_order, text FROM captions WHERE embedding IS NOT NULL")
    while True:
        rows_ = c.fetchmany(batch_size)
        if not rows_:
            break
        for r in rows_:
            yield r
    conn.close()

conn = sqlite3.connect('videos.db', timeout=30)
cursor = conn.cursor()
cursor.execute("SELECT embedding FROM captions WHERE embedding IS NOT NULL LIMIT 1")
first_row = cursor.fetchone()
conn.close()

if not first_row:
    raise ValueError("No caption embeddings found in the database. Compute embeddings first.")

first_embedding = np.array(json.loads(first_row[0])).astype('float32')
dimension = first_embedding.shape[0]

index = faiss.IndexFlatL2(dimension)
metadata_list = []
batch_embeddings = []
batch_metadata = []
bsize = 10000
total_indexed = 0

for row in load_caption_embeddings(bsize):
    cap_id, embedding_json, v_id, chunk_order, text = row
    embedding_vector = np.array(json.loads(embedding_json)).astype('float32')
    batch_embeddings.append(embedding_vector)
    batch_metadata.append({
        'id': cap_id,
        'video_id': v_id,
        'chunk_order': chunk_order,
        'text': text
    })
    if len(batch_embeddings) >= bsize:
        batch_embeddings_np = np.vstack(batch_embeddings)
        index.add(batch_embeddings_np)
        metadata_list.extend(batch_metadata)
        total_indexed += len(batch_embeddings)
        batch_embeddings = []
        batch_metadata = []

if batch_embeddings:
    batch_embeddings_np = np.vstack(batch_embeddings)
    index.add(batch_embeddings_np)
    metadata_list.extend(batch_metadata)
    total_indexed += len(batch_embeddings)

print(f"Total caption embeddings indexed: {total_indexed}")
print(f"FAISS index size: {index.ntotal}")


ERROR: [youtube] w1a9_wYKkqw: Video unavailable. This content isn't available, try again later.
ERROR: [youtube] aqT060DXoNw: Video unavailable. This content isn't available, try again later.
ERROR: [youtube] 3RB6Jn5H5HA: Video unavailable. This content isn't available, try again later.
ERROR: [youtube] YD-9dOqFq1Y: Video unavailable. This content isn't available, try again later.
ERROR: [youtube] 5TQd-FDtqiM: Video unavailable. This content isn't available, try again later.
ERROR: [youtube] 7PcXNyQidrc: Video unavailable. This content isn't available, try again later.
ERROR: [youtube] O9K7GwoL_lI: Video unavailable. This content isn't available, try again later.
ERROR: [youtube] UMhZurFj4Qw: Video unavailable. This content isn't available, try again later.
ERROR: [youtube] YZCsHmzIIG4: Video unavailable. This content isn't available, try again later.
ERROR: [youtube] pj7ap4H7ACs: Video unavailable. This content isn't available, try again later.
ERROR: [youtube] 0MBjcEpCi44: Video unav

Extracted and stored 355 videos with description embeddings.
Could not retrieve transcript for video FPkiKTO5zMg: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=FPkiKTO5zMg! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Could not retrieve transcript for video HEnbbuI_4WA: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=HEnbbuI_4WA! This is most likely caused by:

No transcripts were found for any of the requested language codes: ('en',)

For this video (HEnbbuI_4WA) transcripts are available in th

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Total caption embeddings indexed: 78614
FAISS index size: 78614
